# Importation des données

In [4]:
import os
import string
import re
import math
import statistics
from math import sqrt
import numpy as np
import random
import time
import pandas as pd
import nltk, re, pprint
#nltk.download('punkt')
from nltk import word_tokenize
random.seed(1)
np.random.seed(1)
import matplotlib.pyplot as plt
from IPython.display import clear_output
from matplotlib import pyplot as plt
import collections
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd

import pickle
import datetime

#os.chdir('C:/Users/torna/Documents/StatApp/StatApp')
os.chdir('/Users/alainquartierlatente/Desktop/Ensae/StatApp')
#os.chdir('/home/aqlt/Documents/Ensae/StatApp')
#os.chdir('C:/Users/Kim Antunez/Documents/Projets_autres/StatApp')
nom_dossier = "100k"
#nom_dossier = "ens" #ou

On ne garde ici que les 100 000 premiers tweets

In [5]:
# Penser à changer selon taille
with open("data/%s/vocabulary.file" %nom_dossier, "rb") as f:
    taille_vocab = len(pickle.load(f))

In [6]:
# Fonction pour mettre à jour le graphique en direct
def live_plot(data, figsize=(7,5), title=''):
    clear_output(wait=True)
    plt.figure(figsize=figsize)
    plt.plot(data)
    plt.title(title)
    plt.grid(True)
    plt.xlabel('epoch')
    plt.show();

# Lancement du modèle
Paramètres fixés : la dimension, le nombre de mots tirés dans le *negative sampling* et la proba utilisé, l'affichage du graphique

In [9]:
if nom_dossier == "ens":
    dim = 50
else:
    dim = 20
plot = True
epoch = 100
dim = 100

Paramètres à modifier :

In [10]:
learning_rate = 0.02
window = 4
numero_simulation = 13
seed = 1

In [ ]:
# On crée le dossier Simulation_{numero_simulation}
if not os.path.exists("data/{}/dim{}_ep{}_w{}_lr{}_seed{}".format(nom_dossier, dim, epoch, window, str(learning_rate)[2:], seed)):
    os.mkdir("data/{}/dim{}_ep{}_w{}_lr{}_seed{}".format(nom_dossier, dim, epoch, window, str(learning_rate)[2:], seed))
else:
    print("Attention : le dossier data/{}/dim{}_ep{}_w{}_lr{}_seed{} existe déjà".format(nom_dossier, dim, epoch, window, str(learning_rate)[2:], seed))

torch.manual_seed(seed)
input = torch.randn(taille_vocab, dim)
output = torch.randn(taille_vocab, dim)
input = autograd.Variable(input, requires_grad=True)
output = autograd.Variable(output, requires_grad=True)

loss_tot = []
temps_par_epoch = []

start = time.time()
for i in range(epoch):
    loss_val = compteur = 0
    start_epoch = time.time()
    
    currentDT = datetime.datetime.now()
    print("Epoch {} à {}:{} - Lecture du fichier data/{}/window_{}/epoch_{}.file".format(i, currentDT.hour,currentDT.minute, nom_dossier, window, i))
    with open("data/%s/window_%i/epoch_%i.file" % (nom_dossier, window, i), "rb") as f:
        test_sample = pickle.load(f)

    for focus, context, neg_sample in test_sample:
        compteur = compteur + 1
        if(compteur % 10 == 0 and nom_dossier == "ens"):
            currentDT = datetime.datetime.now()
            print("Tweet %i à %d:%d" % (compteur, currentDT.hour,currentDT.minute))


        data = torch.matmul(input[focus,], torch.t(output[context,]))
        loss1 = - F.logsigmoid(data)

        data = torch.matmul(input[focus,], torch.t(output[neg_sample,]))
        loss2 = - F.logsigmoid(-data).sum()
        loss_val += loss1 + loss2
        # Pour ensuite dériver les matrices par rapport à la loss
        (loss1+loss2).backward()

        # Il faut modifier juste le .data pour ne pas perdre la structure
        input.data = input.data - learning_rate * input.grad.data
        output.data = output.data - learning_rate * output.grad.data

        input.grad.data.zero_()
        output.grad.data.zero_()

    with open("data/{}/dim{}_ep{}_w{}_lr{}_seed{}/input_{}.file".format(nom_dossier, dim, epoch, window, str(learning_rate)[2:], seed, (len(loss_tot)+1)), "wb") as f:
        pickle.dump(input, f, pickle.HIGHEST_PROTOCOL)
    with open("data/{}/dim{}_ep{}_w{}_lr{}_seed{}/output_{}.file".format(nom_dossier, dim, epoch, window, str(learning_rate)[2:], seed, (len(loss_tot)+1)), "wb") as f:
        pickle.dump(output, f, pickle.HIGHEST_PROTOCOL)
    with open("data/{}/dim{}_ep{}_w{}_lr{}_seed{}/loss.file".format(nom_dossier, dim, epoch, window, str(learning_rate)[2:], seed), "wb") as f:
        pickle.dump(loss_tot, f, pickle.HIGHEST_PROTOCOL)
    with open("data/{}/dim{}_ep{}_w{}_lr{}_seed{}/temps_par_epoch.file".format(nom_dossier, dim, epoch, window, str(learning_rate)[2:], seed), "wb") as f:
        pickle.dump(temps_par_epoch, f, pickle.HIGHEST_PROTOCOL)
        
    end_epoch = time.time()
    temps_par_epoch.append(end_epoch - start_epoch)
    loss_val = loss_val / taille_vocab
    loss_tot.append(loss_val)
    if plot:
        live_plot(loss_tot)
    print(round((end_epoch - start_epoch)/60, 2))
end = time.time()
print(round((end - start)/60, 2))
print(statistics.mean(temps_par_epoch)/60)

Attention : le dossier data/100k/dim100_ep100_w4_lr02_seed1 existe déjà
Epoch 0 à 21:39 - Lecture du fichier data/100k/window_4/epoch_0.file


In [27]:
with open("data/{}/gensim/dim{}_ep{}_w{}_lr{}_seed{}/input_{}.file".format(nom_dossier, dim, epoch, window, str(learning_rate)[2:], seed, (len(loss_tot)+1)), "rb") as f:
    input = pickle.load(f)
with open("data/{}/gensim/dim{}_ep{}_w{}_lr{}_seed{}/output_{}.file".format(nom_dossier, dim, epoch, window, str(learning_rate)[2:], seed, (len(loss_tot)+1)), "rb") as f:
    output = pickle.load(f)
with open("data/{}/gensim/dim{}_ep{}_w{}_lr{}_seed{}/loss.file".format(, dim, epoch, window, str(learning_rate)[2:], seed), "rb") as f:
    loss_tot = pickle.load(f)
with open("data/{}/gensim/dim{}_ep{}_w{}_lr{}_seed{}/temps_par_epoch.file".format(, dim, epoch, window, str(learning_rate)[2:], seed), "rb") as f:
    temps_par_epoch = pickle.load(f)

tensor(False)


In [12]:
torch.manual_seed(1)
input = torch.randn(taille_vocab, dim)
torch.manual_seed(1)
input2 = torch.randn(taille_vocab, dim)
print(torch.all(torch.eq(input, input2)))

tensor(True)


In [10]:
dim

20